# Fetch Articles Abstracts from Scopus Search API

### by Miguel Silva Brito

### Dec, 2017

First you need an API KEY from Scopus, get it from https://dev.elsevier.com

Remember to connect to your University VPN first!


In [ ]:
API_KEY=""##### <<<<<<<< CHANGE HERE!!!!

In [108]:
#max number of results to fetch each time depending of Service Level in Scopus API 
COUNT=5

Your query in here

In [109]:
QUERY="MACHINE LEARNING FORECAST DEMAND "

In [110]:
import requests
import json
import pandas as pd

### Search request to Scopus API

In [111]:
def run_scopus_search(start):
    resp = requests.get("http://api.elsevier.com/content/search/scopus?start=%s&count=%s&query=%s" % (str(start),str(COUNT),QUERY),
                        headers={'Accept':'application/json','X-ELS-APIKey': API_KEY})
    print("fetching Scopus IDs... %d " % start)
    results= json.loads(resp.text.encode('utf-8'))
    return results

### Return scopus IDs from results 

In [112]:
def get_scopus_ids(results):
    return [[str(r['dc:identifier'])] for r in results['search-results']["entry"]]

### Return total numbers of articles found 

In [113]:
def get_totalResults(results):
    return int(results['search-results']['opensearch:totalResults'])

### Return basic info about article

In [114]:
def get_scopus_info(SCOPUS_ID):
    url = ("https://api.elsevier.com/content/abstract/scopus_id/"+ SCOPUS_ID + "?view=FULL")
    resp = requests.get(url,headers={'Accept':'application/json','X-ELS-APIKey': API_KEY})

    return json.loads(resp.text.encode('utf-8'))

### Return abstract article

In [115]:
def get_scopus_text(SCOPUS_ID):
    url = ("https://api.elsevier.com/content/article/scopus_id/" + SCOPUS_ID + "?view=FULL")
    resp = requests.get(url,headers={'Accept':'application/json','X-ELS-APIKey': API_KEY})
    return json.loads(resp.text.encode('utf-8'))

because fetching results could take hours we store results in base file in case the process is interrupt, you can start it again without fetch again the abstracts

In [134]:
from pathlib import Path
number_articles_allready_load=0
base = Path("./data/scopus_search_base.csv")
base_exist=False
new_base_exist=False
if base.is_file():
    article_set_readed=pd.read_csv("./data/scopus_search_base.csv")
    base_exist=True
    number_articles_allready_load=article_set.shape[0]
    print('base exists ,%d articles '% number_articles_allready_load) 
new_base_exist=base_exist

### Main

run scopus search, return top 200 and number of total articles
loop by next 200 until reach total articles

write dataset to csv file

In [ ]:
start=0
article_data=[]

n=0

results=run_scopus_search(start)
totalResults=get_totalResults(results)

scopus_ids=get_scopus_ids(results)
if base_exist==True:
    scopus_idsdf=pd.DataFrame(scopus_ids)
    scopus_ids=scopus_idsdf[~scopus_idsdf.isin(article_set_readed['Scopus_ID'])].dropna(how='all').values

n_ids_fetched =len(scopus_ids)
print("number of scopus ids fetched : %d de %d" % (n_ids_fetched,totalResults))

while (n<totalResults-number_articles_allready_load):
    nn=0
    for scopus_id in scopus_ids:

        results_info = get_scopus_info(scopus_id[0])
        results_text = get_scopus_text(scopus_id[0])
        try:
            title=results_info['abstracts-retrieval-response']['coredata']['dc:title']
            pubDate=results_info['abstracts-retrieval-response']['coredata']['prism:coverDate']
            try:
                abstract=str(results_text['full-text-retrieval-response']['coredata']['dc:description'])
                abstract=abstract.replace('Abstract\n               \n                  ','')
            except:
                abstract='(not available)'
            print("fetching %d / %d..." % (n+1,totalResults)) 
            n=n+1
            nn=nn+1
            author=[[str(row['ce:indexed-name'])] for row in results_info['abstracts-retrieval-response']['coredata']['dc:creator']['author']]
            article_data.append({'Scopus_ID' : scopus_id[0],'title':title,'pubDate':pubDate,'abstract': abstract,'author': author[0][0]})
        except:
            pass
    if nn>0:
        article_set=pd.DataFrame(article_data)
        article_data=[]
        if (new_base_exist==False):
            print('creating base')
            article_set.to_csv("./data/scopus_search_base.csv",sep=';',index=False)
            new_base_exist=True
        else:
            print('append to base')
            article_set.to_csv("./data/scopus_search_base.csv",mode="a",sep=';',header=False,index=False)
           
    start=start+n_ids_fetched
    results=run_scopus_search(start)
    scopus_ids=get_scopus_ids(results)
    n_ids_fetched =len(scopus_ids)
    if base_exist==True:
        scopus_idsdf=pd.DataFrame(scopus_ids)
        scopus_ids=scopus_idsdf[~scopus_idsdf.isin(article_set_readed['Scopus_ID'])].dropna(how='all').values
        print('novos artigos= %d %d' % (len(scopus_ids),article_set_readed.shape[0]))
    print("number of scopus ids fetched : %d de %d" % (n_ids_fetched+start,totalResults))
print("Finish!")

fetching Scopus IDs... 0 
number of scopus ids fetched : 5 de 4600
fetching 1 / 4600...
fetching 2 / 4600...
fetching 3 / 4600...
fetching 4 / 4600...
fetching 5 / 4600...
creating base
fetching Scopus IDs... 5 
[['SCOPUS_ID:85036466647'], ['SCOPUS_ID:85034648827'], ['SCOPUS_ID:85032699626'], ['SCOPUS_ID:85038823053'], ['SCOPUS_ID:85037706900']]
number of scopus ids fetched : 10 de 4600
fetching 6 / 4600...
fetching 7 / 4600...
fetching 8 / 4600...
fetching 9 / 4600...
fetching 10 / 4600...
append to base
fetching Scopus IDs... 10 
[['SCOPUS_ID:85038106127'], ['SCOPUS_ID:85037134079'], ['SCOPUS_ID:85030096149'], ['SCOPUS_ID:85034661580'], ['SCOPUS_ID:85030764780']]
number of scopus ids fetched : 15 de 4600
fetching 11 / 4600...
fetching 12 / 4600...
fetching 13 / 4600...
fetching 14 / 4600...
fetching 15 / 4600...
append to base
fetching Scopus IDs... 15 
[['SCOPUS_ID:85037983717'], ['SCOPUS_ID:85037057898'], ['SCOPUS_ID:85038860726'], ['SCOPUS_ID:85037716726'], ['SCOPUS_ID:85036461446

In [ ]:
article_set=pd.read_csv("./data/scopus_search_base.csv") 
print(article_set.info)
print(article_set.head)